In [ ]:
!pip install --quiet langchain-core==0.3.59 langgraph==0.5.3 langchain-openai==0.3.16 langchain-experimental==0.3.4 pydantic==2.11.9

from typing import Annotated
from langchain_core.tools import tool
import pandas as pd
import os
from langchain_experimental.utilities import PythonREPL
import requests


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Tools

In [ ]:
@tool
def fpl_data_tool(
    query: Annotated[str, "The FPL data to retrieve. Options are 'players', 'teams', 'fixtures', 'events'."],
):
    """Use this to retrieve FPL data from the official FPL API."""
    base_url = "https://fantasy.premierleague.com/api/"
    endpoints = {
        "players": "bootstrap-static/",
        "teams": "teams/",
        "fixtures": "fixtures/",
        "events": "events/",
    }
    if query not in endpoints:
        return f"Invalid query. Please choose from {list(endpoints.keys())}."
    try:
        response = requests.get(base_url + endpoints[query])
        response.raise_for_status()
        data = response.json()
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed:\nFPL {query} data retrieved with {len(data)} records."


repl = PythonREPL()

@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user. The chart should be displayed using `plt.show()`."""
    try:
        result = repl.run(code)
    except BaseException as e:
        return f"Failed to execute. Error: {repr(e)}"
    return f"Successfully executed the Python REPL tool.\n\nPython code executed:\n\`\`\`python\n{code}\n\`\`\`\n\nCode output:\n\`\`\`\n{result}\`\`\`"